In [1]:
import torch
print(torch.__version__)

2.5.1+cu118


In [2]:
class NeuralNetwork(torch.nn.Module):
    def __init__(self, num_inputs, num_outputs):
        super().__init__()
        
        self.layers = torch.nn.Sequential(
            torch.nn.Linear(num_inputs, 50),
            torch.nn.ReLU(),
            torch.nn.Linear(50, 30),
            torch.nn.ReLU(),
            torch.nn.Linear(30, num_outputs),
        )
        
    def forward(self, x):
        logits = self.layers(x)
        return logits 

In [3]:
from torch.utils.data import Dataset, DataLoader

class ToyDataset(Dataset):
    def __init__(self, X, y):
        self.features = X
        self.labels = y
        
    def __getitem__(self, index):
        one_X = self.features[index]
        one_y = self.labels[index]
        return one_X, one_y
    
    def __len__(self):
        return self.labels.shape[0]    

In [4]:
x_train = torch.tensor([
    [1.0, 2.2],
    [3.4, -0.5],
    [4.3, 6.0], 
    [1.2, 3.9], 
    [-0.2, 2.4], 
    [3.5, -7.7],
])

y_train = torch.tensor([0, 0, 0, 1, 1, 1])

x_test = torch.tensor([
    [1.0, 2.2],
    [3.4, -0.5],
    [-0.2, 2.4], 
    [3.5, -7.7],
    ])

y_test = torch.tensor([0,0,1,1])

train_ds = ToyDataset(x_train, y_train)
test_ds = ToyDataset(x_test, y_test)    

In [5]:
train_dataloader = DataLoader(
    batch_size = 3,
    shuffle = False,
    dataset = train_ds,
    num_workers = 0
)

test_dataloader = DataLoader(
    batch_size = 3,
    shuffle = False,
    dataset = test_ds,
    num_workers = 0
)

In [6]:
for idx, (x, y) in enumerate(train_dataloader):
    print(f"batch {idx+1}:", x, y)
    
for idx, (x, y) in enumerate(test_dataloader):
    print(f"batch {idx+1}:", x, y)

batch 1: tensor([[ 1.0000,  2.2000],
        [ 3.4000, -0.5000],
        [ 4.3000,  6.0000]]) tensor([0, 0, 0])
batch 2: tensor([[ 1.2000,  3.9000],
        [-0.2000,  2.4000],
        [ 3.5000, -7.7000]]) tensor([1, 1, 1])
batch 1: tensor([[ 1.0000,  2.2000],
        [ 3.4000, -0.5000],
        [-0.2000,  2.4000]]) tensor([0, 0, 1])
batch 2: tensor([[ 3.5000, -7.7000]]) tensor([1])


In [8]:
import torch.nn.functional as F

torch.manual_seed(123)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

model = NeuralNetwork(num_inputs=2, num_outputs=2).to(device)
optimizer = torch.optim.SGD(model.parameters(), lr = 0.05)
num_epoch = 3

for num in range(num_epoch):
    model.train()
    for idx, (x, y) in enumerate(train_dataloader):
        
        x = x.to(device)
        y = y.to(device)
        
        logits = model(x)
        loss = F.cross_entropy(logits, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        print(f"train loss: {loss:.2f}") 
    
    model.eval()   
    correct = 0.0
    total_ex = 0
    with torch.no_grad():
        for idx, (x, y) in enumerate(test_dataloader):
                    
            x = x.to(device)
            y = y.to(device)
        
            logits = model(x)
            loss = F.cross_entropy(logits, y)
            
            print(f"eval loss: {loss:.2f}") 
            
            prediction = torch.argmax(logits, dim=1)
            compare = prediction == y
            correct += torch.sum(compare)
            total_ex += len(compare)
            
            print(f"accuracy: {(correct/total_ex).item():2f}")
            

        
        


cuda
train loss: 0.75
train loss: 0.99
eval loss: 0.70
accuracy: 0.333333
eval loss: 0.60
accuracy: 0.500000
train loss: 0.69
train loss: 0.79
eval loss: 0.67
accuracy: 0.666667
eval loss: 0.40
accuracy: 0.750000
train loss: 0.63
train loss: 0.70
eval loss: 0.65
accuracy: 0.666667
eval loss: 0.31
accuracy: 0.750000
